## Erstmal auf den Zyklus gruppieren und dann Features erstellen

## Forschungsfrage:
- Welche aktuelle und vorherige Verwendung haben wie viel Einfluss auf die aktuelle Kapazität der Batterie?


### Import

In [1]:
import pandas as pd
import numpy as np

import os
import sys
import logging

import functions as fu
from feature_engineering import *

# Pfade setzen
c_fold = fu.get_folder()
p_fold = c_fold + '\data\Prepared'
folder = c_fold + '\data\Processed\Zyklus'
folder_final = c_fold + '\data\Processed\\final'


# Initialize Logging
logging.basicConfig(format='[%(asctime)s] %(levelname)s: %(message)s',
                    datefmt='%H:%M:%S',
                    level=logging.INFO,
                    handlers=[
                        logging.FileHandler(
                            filename='datalogger.log',
                            mode='a',
                            encoding='utf-8'),
                        logging.StreamHandler(sys.stdout)])

In [3]:
def stark_leicht(type_, current_mean, relativeTime_amax):
    entladen_stark = 0
    entladen_leicht = 0
    laden_stark = 0
    laden_leicht = 0
    pause = 0
    
    
    if type_ == 'D': #discharge
        if current_mean >= 3:
            entladen_stark = relativeTime_amax
        else:
            entladen_leicht = relativeTime_amax
    if type_ == 'C': #charge
        if current_mean <= -3:
            laden_stark = relativeTime_amax
        else:
            laden_leicht = relativeTime_amax
    if type_ == 'R':
        pause = relativeTime_amax
    return entladen_stark, entladen_leicht, laden_stark, laden_leicht, pause

    

def temp_hoch(temperature_amax, relativeTime_amax):
    temp_hoch = 0
    if temperature_amax >= 30:
        return relativeTime_amax
    else:
        return 0
        

for file in os.listdir(p_fold):
    if ".csv" in file:
# file = "prepared_RW1.csv" #für tests nur eins
        logging.info(f"<<< Started processing file: {file} >>>")
        new_filename = file.replace("prepared","processed")

        df = pd.read_csv(f"{p_fold}\\{file}", index_col = 0,
                                 #nrows= 100000
                                )
        logging.info(f"<<< Size of Dataframe: {df.shape} >>>")

        #Auf Ebene Batterie + Zyklus gruppieren mit benötigten Aggregationen
        df_pv = pd.pivot_table(
           df,
           index=['batteryname','zyklus','comment','type'],
           aggfunc={'time': np.min, 'relativeTime':  np.max, 'voltage': [np.min, np.max, np.mean], 'current': [np.min, np.max, np.mean], 'temperature': [np.min, np.max, np.mean]}
        ).reset_index()

        df_pv.columns = ['_'.join(col) for col in list(df_pv.columns)]



        df_pv['time_entladen_stark'],df_pv['time_entladen_leicht'],df_pv['time_laden_stark'],df_pv['time_laden_leicht'],df_pv['time_pause']  = zip(*df_pv.apply(lambda x: stark_leicht(x['type_'],x['current_mean'],x['relativeTime_amax']),axis=1))

        df_pv['time_entladen_stark_vorher'] = df_pv.time_entladen_stark.cumsum()
        df_pv['time_entladen_leicht_vorher'] = df_pv.time_entladen_leicht.cumsum()
        df_pv['time_laden_stark_vorher'] = df_pv.time_laden_stark.cumsum()
        df_pv['time_laden_stark_vorher'] = df_pv.time_laden_leicht.cumsum()
        df_pv['time_pause_vorher'] = df_pv.time_pause.cumsum()


        df_pv['time_temp_hoch'] = df_pv.apply(lambda x: temp_hoch(x['temperature_amax'],x['relativeTime_amax']),axis=1)

        df_pv['time_temp_hoch_vorher'] = df_pv.time_temp_hoch.cumsum()

        df_pv['amperestunden'] = (df_pv['current_mean']*df_pv['relativeTime_amax'])/3600

        df_pv_filtered = df_pv[df_pv['comment_'] == 'reference discharge']

        logging.info(f"<<< Start saving dataframe with shape {df_pv_filtered.shape} >>>")
        df_pv_filtered.to_csv(f"{folder}\\{new_filename}", sep=',', index=False)
        logging.info(f"<<< Saved processed file in directory {folder}\\{new_filename} >>>")

[19:28:54] INFO: <<< Started processing file: prepared_RW1.csv >>>
[19:28:59] INFO: <<< Size of Dataframe: (2958840, 11) >>>
[19:29:01] INFO: <<< Start saving dataframe with shape (48, 27) >>>
[19:29:01] INFO: <<< Saved processed file in directory C:\Users\sebis\OneDrive\Desktop\FOM\3. Semester\Big Data Analyseprojekt\battery_failure_predictions\data\Processed\Zyklus\processed_RW1.csv >>>
[19:29:01] INFO: <<< Started processing file: prepared_RW10.csv >>>
[19:29:15] INFO: <<< Size of Dataframe: (8596025, 11) >>>
[19:29:21] INFO: <<< Start saving dataframe with shape (77, 27) >>>
[19:29:21] INFO: <<< Saved processed file in directory C:\Users\sebis\OneDrive\Desktop\FOM\3. Semester\Big Data Analyseprojekt\battery_failure_predictions\data\Processed\Zyklus\processed_RW10.csv >>>
[19:29:21] INFO: <<< Started processing file: prepared_RW11.csv >>>
[19:29:35] INFO: <<< Size of Dataframe: (8664510, 11) >>>
[19:29:41] INFO: <<< Start saving dataframe with shape (77, 27) >>>
[19:29:41] INFO: <<<

## DFs zusammenführen

In [103]:
d = []

df_full = pd.DataFrame(d, columns=df_pv_filtered.columns)

for file in os.listdir(folder):
    if ".csv" in file:
        df = pd.read_csv(f"{folder}\\{file}")
        df_full = df_full.append(df)
df_full = df_full.reset_index()

# Variablenauswahl
* **index** - kann raus
* **batteryname_** - kann weg: keinen Einfluss
* **zyklus_** - bleibt: Anzahl an Lade/Entlade/Pause Wechseln
* **comment_** - kann weg: ist in time_entladen_*
* **type_** - kann weg: ist in time_entladen_*
* **current_amax** - kann weg: ist in Zielvariable/immer gleich
* **current_amin** - kann weg: ist in Zielvariable/immer gleich
* **current_mean** - kann weg: ist in Zielvariable/immer gleich
* **relativeTime_amax** - kann weg: ist in Zielvariable
* **temperature_amax** - bleibt: kann man mal ausprobieren, ob Temp Einfluss hat
* **temperature_amin** - bleibt: kann man mal ausprobieren, ob Temp Einfluss hat
* **temperature_mean** - bleibt: kann man mal ausprobieren, ob Temp Einfluss hat
* **time_amin** - bleibt: Gesamtalter der Batterie bei Start
* **voltage_amax** - kann weg: bei Ref Discharge immer von 4.2V zu 3.2V
* **voltage_amin** - kann weg: bei Ref Discharge immer von 4.2V zu 3.2V
* **voltage_mean** - kann weg: bei Ref Discharge immer von 4.2V zu 3.2V
* **time_entladen_stark** - kann weg: nur benötigt für "vorher"-Berechnung
* **time_entladen_leicht** - kann weg: nur benötigt für "vorher"-Berechnung
* **time_laden_stark** - kann weg: nur benötigt für "vorher"-Berechnung
* **time_laden_leicht** - kann weg: nur benötigt für "vorher"-Berechnung
* **time_pause** - kann weg: nur benötigt für "vorher"-Berechnung
* **time_entladen_stark_vorher** - bleibt: wie vorher verwendet
* **time_entladen_leicht_vorher** - bleibt: wie vorher verwendet
* **time_laden_stark_vorher** - bleibt: wie vorher verwendet
* **time_pause_vorher** - bleibt: wie vorher verwendet
* **time_temp_hoch** - bleibt: wie vorher verwendet
* **time_temp_hoch_vorher** - bleibt: wie vorher verwendet
* **amperestunden** - bleibt: Zielvariable

In [118]:
var_auswahl = [
    "amperestunden",
    "zyklus_",
    "temperature_amax",
    "temperature_amin",
    "temperature_mean",
    "time_amin",
    "time_entladen_stark_vorher",
    "time_entladen_leicht_vorher",
    "time_laden_stark_vorher",
    "time_pause_vorher",
    "time_temp_hoch",
    "time_temp_hoch_vorher"
]

df_final = df_full[var_auswahl]
df_final.to_csv(f"{folder_final}\\df_fertige_features.csv", sep=',', index=False)
df_final

,amperestunden,zyklus_,temperature_amax,temperature_amin,temperature_mean,time_amin,time_entladen_stark_vorher,time_entladen_leicht_vorher,time_laden_stark_vorher,time_pause_vorher,time_temp_hoch,time_temp_hoch_vorher
0,2.000364,3,23.56742,18.39025,21.798768,385232.92,0.00,196178.50,194718.28,0.00,0.00,0.000000e+00
1,2.000250,5,24.06639,18.15403,21.449898,405533.19,0.00,203379.98,207816.73,0.00,0.00,0.000000e+00
2,1.958481,1009,22.81896,17.57133,20.530506,839519.21,17749.62,312669.57,472156.60,16048.39,0.00,1.784186e+05
3,1.963192,1011,23.64538,18.12253,21.420698,858525.62,17749.62,319737.63,484111.86,16048.39,0.00,1.784186e+05
4,1.911217,1925,23.89487,17.66582,20.839432,1248403.47,36155.14,406768.88,743215.33,28472.48,0.00,3.443131e+05
...,...,...,...,...,...,...,...,...,...,...,...,...
945,0.784939,107540,37.88939,31.96866,35.245658,12207354.10,315859.21,4021813.10,4949765.16,1036683.27,2826.01,9.016036e+06
946,0.759776,110551,37.79663,32.10779,35.343405,12427168.28,317601.49,4080113.57,5040620.52,1059934.58,2735.55,9.190943e+06
947,0.758740,110555,37.53384,31.59765,34.768506,12457189.16,317601.49,4082845.38,5060405.85,1067434.58,2731.81,9.220960e+06
948,0.748778,113572,38.29132,32.40151,35.719393,12672750.22,319549.85,4142731.52,5144349.95,1094276.63,2695.90,9.394390e+06


## Nächste Schritte - Fokus Datenanalyse:
* Visulisierung des Kapazitätsverlusts
* Analyse welche vorherige Verwendung besonders Einfluss auf die Kapazität hat